In [ ]:
import itertools
import json
import logging
import os
import pickle
import unittest
from contextlib import contextmanager
from operator import itemgetter
from random import sample

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pymatgen
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from pandas.testing import assert_frame_equal, assert_series_equal
from pymatgen.analysis.structure_prediction.substitution_probability import SubstitutionProbability
from pymatgen.analysis.local_env import CrystalNN


import smact
from smact import Species
from smact.structure_prediction import prediction, database, mutation, probability_models, structure, utilities
from smact.structure_prediction.database import StructureDB
from smact.structure_prediction.mutation import CationMutator
from smact.structure_prediction.prediction import StructurePredictor
from smact.structure_prediction.structure import SmactStructure
from smact.structure_prediction.utilities import parse_spec, unparse_spec
from pymatgen.analysis import structure_matcher
import smact
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Composition
import pandas as pd

from smact import Element,element_dictionary, ordered_elements, neutral_ratios
from smact.screening import smact_filter, pauling_test

from datetime import datetime
from pathos.multiprocessing import ProcessPool as Pool

m = MPRester("Z51OJasyeWwk4pci")

In [2]:
#query the MP database for binary materials with 1:1 stoichiometry
anon_formula = {'A':1, 'B':1}
binary = m.query(criteria = {'anonymous_formula':anon_formula, 'icsd_ids':{'$gte':0}}, properties = ['pretty_formula', 'material_id', 'spacegroup.symbol', 'spacegroup.crystal_system', 'icsd_ids', 'e_above_hull', 'band_gap', 'structure'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2201/2201 [00:08<00:00, 267.13it/s]


In [3]:
binary_df = pd.DataFrame(binary)
binary_df = binary_df.sort_values(['pretty_formula', 'e_above_hull'])
binary_df = binary_df.reset_index(drop = True)
binary_df = binary_df.drop_duplicates('pretty_formula', keep = 'first').reset_index(drop=True)
binary_data = binary_df.to_dict('records')

In [4]:
SM=structure_matcher.StructureMatcher(attempt_supercell=True)

In [5]:
rock_salt = m.query('mp-22862', properties=['pretty_formula', 'material_id', 'spacegroup.symbol', 'spacegroup.crystal_system', 'icsd_ids', 'e_above_hull', 'band_gap', 'structure'])
rock_salt_structure = rock_salt[0]['structure']

In [6]:
fitted_data = []
for i in binary_data:
    if SM.fit_anonymous(i['structure'],rock_salt_structure):
        fitted_data.append(i)
print(len(fitted_data))

299


In [7]:
cscl = m.query('mp-22865', properties=['pretty_formula', 'material_id', 'spacegroup.symbol', 'spacegroup.crystal_system', 'icsd_ids', 'e_above_hull', 'band_gap', 'structure'])
cscl_structure = cscl[0]['structure']

In [8]:
for i in binary_data:
    if SM.fit_anonymous(i['structure'],cscl_structure):
        fitted_data.append(i)
print(len(fitted_data))

618


In [9]:
sphalerite = m.query('mp-10695', properties=['pretty_formula', 'material_id', 'spacegroup.symbol', 'spacegroup.crystal_system', 'icsd_ids', 'e_above_hull', 'band_gap', 'structure'])
sphalerite_structure = sphalerite[0]['structure']

In [10]:
for i in binary_data:
    if SM.fit_anonymous(i['structure'],sphalerite_structure):
        fitted_data.append(i)
print(len(fitted_data))

677


In [11]:
wurtzite = m.query('mp-560588', properties=['pretty_formula', 'material_id', 'spacegroup.symbol', 'spacegroup.crystal_system', 'icsd_ids', 'e_above_hull', 'band_gap', 'structure'])
wurtzite_structure = wurtzite[0]['structure']

In [12]:
for i in binary_data:
    if SM.fit_anonymous(i['structure'],wurtzite_structure):
        fitted_data.append(i)
print(len(fitted_data))

690


In [15]:
fitted_data

[{'pretty_formula': 'AgBr',
  'material_id': 'mp-23231',
  'spacegroup.symbol': 'Fm-3m',
  'spacegroup.crystal_system': 'cubic',
  'icsd_ids': [56548, 56547, 65061, 53850, 56546, 52246, 65062, 157536],
  'e_above_hull': 0.03729415000000014,
  'band_gap': 0.7946,
  'structure': Structure Summary
  Lattice
      abc : 4.1377258397810746 4.1377258397810746 4.1377258397810746
   angles : 60.00000000000001 60.00000000000001 60.00000000000001
   volume : 50.0922035521385
        A : 0.0 2.925814 2.925814
        B : 2.925814 0.0 2.925814
        C : 2.925814 2.925814 0.0
      pbc : True True True
  PeriodicSite: Ag (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
  PeriodicSite: Br (2.9258, 2.9258, 2.9258) [0.5000, 0.5000, 0.5000]},
 {'pretty_formula': 'AgC',
  'material_id': 'mp-1008653',
  'spacegroup.symbol': 'Fm-3m',
  'spacegroup.crystal_system': 'cubic',
  'icsd_ids': [183175],
  'e_above_hull': 2.235510398333333,
  'band_gap': 0.0,
  'structure': Structure Summary
  Lattice
      ab

In [16]:
fitted_df = pd.DataFrame(fitted_data)
fitted_df

,pretty_formula,material_id,spacegroup.symbol,spacegroup.crystal_system,icsd_ids,e_above_hull,band_gap,structure
0,AgBr,mp-23231,Fm-3m,cubic,"[56548, 56547, 65061, 53850, 56546, 52246, 650...",0.037294,0.7946,"[[0. 0. 0.] Ag, [2.925814 2.925814 2.925814] Br]"
1,AgC,mp-1008653,Fm-3m,cubic,[183175],2.235510,0.0000,"[[0. 0. 0.] Ag, [2.333879 2.333879 2.333879] C]"
2,AgCl,mp-22922,Fm-3m,cubic,"[56540, 56539, 157535, 56538, 64734]",0.000000,1.2003,"[[0. 0. 0.] Ag, [2.811198 2.811198 2.811198] Cl]"
3,AgF,mp-7592,Fm-3m,cubic,[18008],0.000000,0.0000,"[[0. 0. 0.] Ag, [2.514648 2.514648 2.514648] F]"
4,AgN,mp-1056955,Fm-3m,cubic,"[167858, 185555, 183195]",1.408179,0.0000,"[[0. 0. 0.] Ag, [2.30808 2.30808 2.30808] N]"
...,...,...,...,...,...,...,...,...
685,MgTe,mp-1039,P6_3mc,hexagonal,"[52363, 642882, 168348, 642883]",0.000000,2.3620,"[[2.30544 1.33104933 7.48602835] Mg, [ 2.30..."
686,MnSe,mp-999540,P6_3mc,hexagonal,[643594],0.000000,0.0000,"[[ 2.102293 -1.21376176 3.41882115] Mn, [2...."
687,TlN,mp-1017982,P6_3mc,hexagonal,"[644794, 186556]",0.721442,0.0000,"[[ 1.864724 -1.07660082 3.0409322 ] Tl, [1...."
688,ZnBi,mp-1064405,P6_3mc,hexagonal,[100115],0.242919,0.0000,"[[2.353272 1.35866505 3.41544156] Zn, [ 2.35..."


In [17]:
fitted_df.to_csv('data_filtered_structs.csv',index=False)

In [18]:
#This creates the database object
DB=database.StructureDB("Binary_structures_filtered.db")

#These create tables within the database
DB.add_table("Experimental")

In [19]:
structs=[]
for i in fitted_data:
    structs.append(database.parse_mprest(i))

Couldn't decorate mp-8023 with oxidation states.
Couldn't decorate mp-1008610 with oxidation states.
Couldn't decorate mp-1057758 with oxidation states.
Couldn't decorate mp-1058549 with oxidation states.
Couldn't decorate mp-1008918 with oxidation states.
Couldn't decorate mp-1025468 with oxidation states.
Couldn't decorate mp-24287 with oxidation states.
Couldn't decorate mp-22907 with oxidation states.
Couldn't decorate mp-1078613 with oxidation states.
Couldn't decorate mp-1025444 with oxidation states.
Couldn't decorate mp-1623 with oxidation states.
Couldn't decorate mp-2491 with oxidation states.
Couldn't decorate mp-1280 with oxidation states.
Couldn't decorate mp-11428 with oxidation states.
Couldn't decorate mp-24289 with oxidation states.
Couldn't decorate mp-1007755 with oxidation states.
Couldn't decorate mp-22866 with oxidation states.
Couldn't decorate mp-1087502 with oxidation states.
Couldn't decorate mp-1079180 with oxidation states.
Couldn't decorate mp-1025438 with 

In [21]:
DB.add_structs(structs, "Experimental")

295

In [29]:
fitted_df['spacegroup.crystal_system'].value_counts()

cubic           568
orthorhombic     71
tetragonal       32
hexagonal        13
trigonal          3
monoclinic        3
Name: spacegroup.crystal_system, dtype: int64

In [30]:
#function to return cation oxidation state
def cat_os(x):
    comp = Composition(x)
    os = comp.oxi_state_guesses()
    if len(os) > 0:
        oxidation_states = list(os[0].values())
    else:
        oxidation_states = [0,0]

    return oxidation_states[0]

#function to return anion oxidation state

def an_os(x):
    comp = Composition(x)
    os = comp.oxi_state_guesses()
    if len(os) > 0:
        oxidation_states = list(os[0].values())
    else:
        oxidation_states = [0,0]

    return oxidation_states[1]

In [31]:
fitted_df['cation_oxidation_state'] = fitted_df['pretty_formula'].apply(cat_os)
fitted_df['anion_oxidation_state'] = fitted_df['pretty_formula'].apply(an_os)
fitted_df

,pretty_formula,material_id,spacegroup.symbol,spacegroup.crystal_system,icsd_ids,e_above_hull,band_gap,structure,cation_oxidation_state,anion_oxidation_state
0,AgBr,mp-23231,Fm-3m,cubic,"[56548, 56547, 65061, 53850, 56546, 52246, 650...",0.037294,0.7946,"[[0. 0. 0.] Ag, [2.925814 2.925814 2.925814] Br]",1.0,-1.0
1,AgC,mp-1008653,Fm-3m,cubic,[183175],2.235510,0.0000,"[[0. 0. 0.] Ag, [2.333879 2.333879 2.333879] C]",2.0,-2.0
2,AgCl,mp-22922,Fm-3m,cubic,"[56540, 56539, 157535, 56538, 64734]",0.000000,1.2003,"[[0. 0. 0.] Ag, [2.811198 2.811198 2.811198] Cl]",1.0,-1.0
3,AgF,mp-7592,Fm-3m,cubic,[18008],0.000000,0.0000,"[[0. 0. 0.] Ag, [2.514648 2.514648 2.514648] F]",1.0,-1.0
4,AgN,mp-1056955,Fm-3m,cubic,"[167858, 185555, 183195]",1.408179,0.0000,"[[0. 0. 0.] Ag, [2.30808 2.30808 2.30808] N]",3.0,-3.0
...,...,...,...,...,...,...,...,...,...,...
685,MgTe,mp-1039,P6_3mc,hexagonal,"[52363, 642882, 168348, 642883]",0.000000,2.3620,"[[2.30544 1.33104933 7.48602835] Mg, [ 2.30...",2.0,-2.0
686,MnSe,mp-999540,P6_3mc,hexagonal,[643594],0.000000,0.0000,"[[ 2.102293 -1.21376176 3.41882115] Mn, [2....",2.0,-2.0
687,TlN,mp-1017982,P6_3mc,hexagonal,"[644794, 186556]",0.721442,0.0000,"[[ 1.864724 -1.07660082 3.0409322 ] Tl, [1....",3.0,-3.0
688,ZnBi,mp-1064405,P6_3mc,hexagonal,[100115],0.242919,0.0000,"[[2.353272 1.35866505 3.41544156] Zn, [ 2.35...",0.0,0.0


In [32]:
#dropping the compositions with no oxidation states (cation_oxidation_state = 0)
fitted_df = fitted_df[fitted_df.cation_oxidation_state != 0]
fitted_df = fitted_df.reset_index(drop=True)

In [33]:
#function to estimate coordination numbers
from pymatgen.analysis.local_env import CrystalNN


def co_ordination_num(y):
    structure = m.get_structure_by_material_id(y)
    crystal = CrystalNN()
    cn = crystal.get_cn(structure, 0)
    return cn


from tqdm import tqdm

tqdm.pandas()
fitted_df['coordination_number'] = fitted_df['material_id'].progress_apply(co_ordination_num)

  0%|                                                                                                                                                                                                                          | 0/290 [00:00<?, ?it/s]C:\Users\ashis\AppData\Local\Programs\Python\Python310\lib\site-packages\pymatgen\analysis\local_env.py:4121: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
C:\Users\ashis\AppData\Local\Programs\Python\Python310\lib\site-packages\pymatgen\analysis\local_env.py:3924: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 290/290 [05:41<00:00,  1.

In [35]:
#create a new dataframe for radius ratio rules
fitted_df_new = fitted_df.copy(deep=True)

#function to return radius ratio and corresponding coordination number and geometry
def radius_ratio(x):
    comp = Composition(x['pretty_formula'])
    elements = comp.chemical_system
    #split the individual elements in the system
    i_element = elements.partition('-')

    #create cation and anion species using smact
    cat = smact.Species(i_element[0], x['cation_oxidation_state'])
    an = smact.Species(i_element[2], x['anion_oxidation_state'])

    #calculate radius ratio
    rad_ratio = cat.average_ionic_radius/an.average_ionic_radius

    return rad_ratio

In [36]:
fitted_df_new['radius_ratio'] = fitted_df_new.dropna().apply(radius_ratio, axis=1)
fitted_df_new.head()

,pretty_formula,material_id,spacegroup.symbol,spacegroup.crystal_system,icsd_ids,e_above_hull,band_gap,structure,cation_oxidation_state,anion_oxidation_state,coordination_number,radius_ratio
0,AgBr,mp-23231,Fm-3m,cubic,"[56548, 56547, 65061, 53850, 56546, 52246, 650...",0.037294,0.7946,"[[0. 0. 0.] Ag, [2.925814 2.925814 2.925814] Br]",1.0,-1.0,6,0.541545
1,AgC,mp-1008653,Fm-3m,cubic,[183175],2.235510,0.0000,"[[0. 0. 0.] Ag, [2.333879 2.333879 2.333879] C]",2.0,-2.0,6,NaN
2,AgCl,mp-22922,Fm-3m,cubic,"[56540, 56539, 157535, 56538, 64734]",0.000000,1.2003,"[[0. 0. 0.] Ag, [2.811198 2.811198 2.811198] Cl]",1.0,-1.0,6,0.586425
3,AgF,mp-7592,Fm-3m,cubic,[18008],0.000000,0.0000,"[[0. 0. 0.] Ag, [2.514648 2.514648 2.514648] F]",1.0,-1.0,6,0.812577
4,AgN,mp-1056955,Fm-3m,cubic,"[167858, 185555, 183195]",1.408179,0.0000,"[[0. 0. 0.] Ag, [2.30808 2.30808 2.30808] N]",3.0,-3.0,6,0.486301
